# Software Projekt - Sequence Embeddings on Shallow Learners
**2023, Klaus Hartmann-Baruffi, Fabio Pfaehler**

## o) Open Topics/Questions:
- F: How does our subset look like? Either 1) define a number of protein IDs or 2) define a number of VOGs ?
  1) We would have to collect the subset of protein IDs and convert them into a new fasta file that serves as input for the bio-embedding tool.
  2) We could directly simply define a subset of VOGs and input the corresponding (already VOG assigned) fasta files and could skip all the steps that come before the application of the bio-embeddings tool. <br>
  
  F: I guess rather (1) than (2). I guess it´s more important to cover and prioritize including all the labels and take a subset of protein IDs per VOG. I suggest that the number of proteins chosen per VOG is relative to the size of the VOG. So define the "size of the subset" by choosing a certain percentage value(as the proportion of proteins that will be drawn from each VOG). <br>
  F: Actually taking a percentage might be a bad idea. If I´m not wrong it is cruicial for labels/VOGs with few features, eg. the very last VOG with only 2 protein IDs , that those groups have all IDs presented in our subset.

- F: Further Preprocessing of the input fasta files. <br> 
1) The bio-embeddings module recognizes redundant sequences in our VOGDB fasta files. 
2) The bio-embeddings module takes fasta files as input. So for each VOG=class=label we need a fasta file with a certain subset of Protein sequence entries, we run the module for all our VOGs and get as output the corresponding feature vector (as embedding).

## o) Install Libraries

In [ ]:
"""not tested, taken from one of the provided notebooks found in the bio-embeddings github"""
# # bio-embeddings
# !pip3 install -U pip > /dev/null
# !pip3 install -U bio_embeddings[all] > /dev/null

## o) Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
from Bio import SeqIO
from bio_embeddings.embed import ProtTransBertBFDEmbedder
from bio_embeddings.embed.seqvec_embedder import SeqVecEmbedder
from bio_embeddings.project import tsne_reduce
from bio_embeddings.visualize import render_3D_scatter_plotly

## o) Load Data, Choose a Subset

Our aim is to use shallow learners, hence using the whole dataset (38161 vog groups/instances) is not feasable and we take only a subset to work with.

In [ ]:
# Load data
df = pd.read_csv("/home/dinglemittens/SoftwareProject/VOGDB/vog.members.tsv",sep='\t', header=0)
print("dataset shape: {}\n".format(df.shape))

# Choose subset from vog "start" to vog "end"
start = 5
end = 19
subset = df.iloc[start-1 : end]
print(subset.iloc[:3])

## o) Generate Feature- and Label-Vectors

Number of labels/classes (VOG groups) = size of the subset
Number of features/feature dimensions (protein IDs/sequences) = size of the subset * number of proteins per VOG * length of the proteinsequence * 20 
, where 20 reflects the number of aminoacids in a 1-hot-encoding, since we can´t feed the model with string-characters.

In [ ]:
# Convert unflattened labels (#GroupName) and features (ProteinIDs) into lists
group_names = subset["#GroupName"].tolist()
protein_ids = subset["ProteinIDs"].tolist()

# Generate flattened feature(X)- and label(y)-vectors
X=[]
y=[]
for group in group_names:
    for per_group_ids in protein_ids:
        for protein_id in per_group_ids.split(","): # note: maybe change iterator names (confusing; we have the df ProteinIDs column which contains collections of protein IDs per group, so ProteinIDs contains protein ids)
            y.append(int(group.replace("VOG","")))
            X.append(protein_id)

## o) Generate Bio-Embeddings (in progress)
see <embed_fasta_sequences.ipynb>

As we highlited in the previous step, the dimensions, - complexity of our feature space - , are extraordinary high, we need to reduce the feature size. For this purpose we will use so called protein- or bio-embeddings to ...

In [ ]:
# # Create (download) testing fasta file (tiny_sampled.fasta)
# !wget http://data.bioembeddings.com/public/embeddings/notebooks/custom_data/tiny_sampled.fasta --output-document BE_testing/tiny_sampled.fasta

In [ ]:
# Extract sequences from fasta file and store them as a list
# filepath = "Fabio/BE_testing/tiny_sampled.fasta"
filepath = "BE_testing/VOG1_trial2/VOG00001.faa"
sequences = []
for record in SeqIO.parse(filepath, "fasta"):
    sequences.append(record)

# Sanity-check
print(f"Member-ID     Identifier\t\tLength\t    Sequence\n")
for i,s in enumerate(sequences):
    print(f"Protein {i+1:<6}{(s.id):<28}{len(s.seq):<10}{s.seq}") # :<6 for proper output alignment

sm_86 pytorch compatibility issue:
- https://github.com/pytorch/pytorch/issues/45028
- \\wsl.localhost\Ubuntu\home\dinglemittens\anaconda3\envs\SPEnv38\lib\python3.8\site-packages\bio_embeddings\embed
- solved: conda install pytorch==1.10.0 torchvision==0.11.0 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch -c conda-forge

About SeqVec:
- https://github.com/Rostlab/SeqVec/blob/master/README.md : "All results built upon the embeddings gained from the new tool SeqVec neither explicitly nor implicitly using evolutionary information. Nevertheless, it improved over some methods using such information. Where the lightning-fast HHblits needed on average about two minutes to generate the evolutionary information for a target protein, SeqVec created the vector representation on average in 0.03 seconds."

In [ ]:
# Generate Embedder Object
# embedder = ProtTransBertBFDEmbedder()
embedder = SeqVecEmbedder()

Trouble with ProtTrans Bert:
- Fabio: For the following aa level embedding via ProtTransBertBFDEmbedder() apparently my laptop GPU memory is too low, it uses CPU instead which significantly affects the speed. Interrupted after 40 min. Errormessage:
RuntimeError for sequence with 166 residues: CUDA error: no kernel image is available for execution on the device<br>
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.<br>
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.. This most likely means that you don't have enough GPU RAM to embed a protein this long. Embedding on the CPU instead, which is very slow.


In [ ]:
# Compute Amino Acid Level Embedding (takes quiet some time)
aa_embeddings = embedder.embed_many([str(s.seq) for s in sequences])
# `embed_many` returns a generator.
# We want to keep both RAW embeddings and reduced embeddings in memory.
# To do so, we simply turn the generator into a list!
# (this will start embedding the sequences!)
# Needs certain amount of GPU RAM, if not sufficient CPU is used (slower)
aa_embeddings = list(aa_embeddings)

In [ ]:
# Print Shape of Amino Acid Level Embedding
print("amino acid level embeddings object shape:")
print("o) 3 dimensional list of a number of <len(sequences)> embedding matrices with 1024 rows and <len(seq)> columns") #
print(f"o) 1st D (number of sequences)\t{len(aa_embeddings)}")
print("o) 2nd D (sequence length)\tdepending on sequence")
print(f"o) 3rd D (embedding dimensions)\t{len(aa_embeddings[0][0])} (constant)")

In [ ]:
# Compute Protein Level Embedding
protein_embeddings = [ProtTransBertBFDEmbedder.reduce_per_protein(e) for e in aa_embeddings]
# mean of amino acid level vectors

In [ ]:
# Print Shape of Protein Level Embedding
print("protein level embeddings-object shape:")
print("o) 2 dimensional list of <len(sequences)> embedding vectors with 1024 entries")
print(f"o) 1st D (number of sequences)\t{len(protein_embeddings)}")
print(f"o) 2nd D (embedding dimensions)\t{len(protein_embeddings[0])} (constant)")

In [ ]:
# Print Summary of Embedding Shapes:  Sequence | AA Level Embedding | Protein Level Embedding
for i, (per_amino_acid, per_protein) in enumerate(zip(aa_embeddings, protein_embeddings)):
    print(f"Protein {i+1}\t{per_amino_acid.shape}\t{per_protein.shape}")

## o) Projecting high dimensional embedding space to a 3D space
see <project_visualize_pipeline_embeddings.ipynb> (Bio-embeddings GitHub)


In [ ]:
# Configure tsne options
options = {
    'perplexity': 3, # Low perplexity values (e.g., 3) cause t-SNE to focus more on preserving the local structure of the data (high, e.g. 30).
    'n_iter': 500 # number of iterations for the tsne algorithm
}

# Apply TSNE Projection 
projected_p_embedding = tsne_reduce(protein_embeddings, **options) # list

# Display Projected Embedding (from 1024 dimensional (Protein Level) vectors to 3 dimensional coordinate vectors)
print(f"\nShape of projected protein level embedding: {projected_p_embedding.shape}\n")
for i,embedding in enumerate(projected_p_embedding[:3]): # first 3
    print(f"Protein {i+1}\t{embedding}")
print(". . .")
for i,embedding in enumerate(projected_p_embedding[-3:]): # last 3
    print(f"Protein {i+len(projected_p_embedding)-2}\t{embedding}")

## o) Visualization of the Data
see <project_visualize_pipeline_embeddings.ipynb> (Bio-embeddings GitHub)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Assuming your data is stored in a variable called 'data'
# Replace this with your actual data

# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extract x, y, and z coordinates from the data
x = data[:, 0]
y = data[:, 1]
z = data[:, 2]

# Plot the points
ax.scatter(x, y, z)

# Set labels for each axis
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

# Show the plot
plt.show()

In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA

df = px.data.iris()
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['species'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

## o) Split the Data

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## o) Train a Classifier on the Training Set

In [ ]:
# Define the LDA classifier
"""Add model object"""

# Ttrain the classifier (modelfitting)
"""<model>.fit(X_train, y_train)"""

## o) Prediction on the Validation Set & Accuracy

In [ ]:
# Use your model to make a prediction on the test data
"""y_pred = <model>.predict(X_test)"""

# Compute accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(round(accuracy, 2)))

## o) Visualization/Plot of Decision Boundaries (?)

---
# Older Version of Notebook

In [ ]:
# Step 1: Import the necessary libraries
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Step 2: Load your dataset into a pandas DataFrame
df = pd.read_csv("/home/dinglemittens/SoftwareProject/VOGDB/vog.members.tsv",sep='\t', header=0)
# df = pd.read_csv("VOGDB/test.tsv",sep='\t', header=0)
print(df)

In [ ]:
# Step 3: Preprocess your data
"""Next step is to pick out the relevant categories in my dataframe: The VOG numbers (labels and their 
corresponding collections of ProteinIDs (features). In addition I must convert each ID to it´s sequence by using
the fasta files. 
For the scikit split functoin I need Feature set X and label set y (with redundant labels) of same size.
By now I have my df ordered in such a way that each label has a list of proteins,
but I need the resolve them such that I have a big list of proteins each added with a label.
(Analogy: By now I have containers of balls (proteins/features), I know their label (#VOG/container), 
because they are seperated from other balls through the container. To continue I need to merge all 
the balls of all containers in a pool, before that I label them with the container number. This pool
can now be split 2 : 8 in test and training set. By stratifying (use as parameter) I can inherit the information 
of the frequency distribution of balls from a certain container relative to all balls into the two sets (If all
Ball of container 1 make up 10% of the total number of balls, then in the teset and training set will make up
10% of all balls in each of the two sets)).
Next we don´t want only our features as single strings (sequences) but as numerical vectors, where each
dimension of the vector is an amino acid. The algorithm needs numerical values for learning patterns.
The most straigt forward way would be a 1hot encoding, i.e. one feature would be a vector of vectors of 
length 20, 19 zeros and 1 one (depending on which letter is considered). We won´t do hot1 embedding but another one."""

# select interval for subset (from VOGa to VOGb) 1 - 38.161
end = df.shape[0] # last vog

a = 1
b = 180

features= df['ProteinIDs'].str.split(',').iloc[a-1:b] # each row a VOGs collection of proteins
labels = df['#GroupName'].iloc[a-1:b]

print("features:\n",features, "\n")
print("labels:\n", labels, "\n")

X=[]
y=[]
for i in range(len(features)): # for each VOG
    # id2seqvec = vog2fasta_dict(labels[i])
    for j in range(len(features.iloc[i])): # for each VOGs proteinIDs
        y.append(labels[i])
        X.append("add function here that turns ProteinID into sequence embedding")

print("X:\n",X[:8], "...\n")
print("y:\n",y[:8], "...\n")

In [ ]:
# Step 4: Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train:\n", X_train[:8], "...\n")
print("y_train:\n", y_train[:8], "...\n")
print("X_test:\n", X_test[:8], "...\n")
print("y_test:\n", y_test[:8], "...\n")







In [ ]:
# Step 5: Choose a machine learning algorithm to use
model = LogisticRegression()

In [ ]:

# Step 6: Train the model on the training data
model.fit(X_train, y_train)

In [ ]:
proteinids = X_train.loc[:, 'ProteinIDs']
new_df = pd.DataFrame({'ProteinIDs': proteinids})
new_df.to_excel('./vog_proteins.xlsx', index=False)

new_df = new_df['ProteinIDs'].str.split(',', expand=True)
print(new_df)


In [ ]:

# Step 7: Evaluate the model's performance on the testing data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
# Step 8: Tune the model's hyperparameters to improve its performance
# For example, you could use GridSearchCV to search over a range of hyperparameters

In [ ]:

# Step 9: Use the model to make predictions on new data
# For example, you could use model.predict(new_data) to make predictions on new data